In [2]:
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import pandas as pd 
import numpy as np
from fastFM import sgd
from scipy import sparse
from scipy.sparse import csc_matrix
import time
from sklearn.metrics import auc,accuracy_score,roc_curve,roc_auc_score
from sklearn.metrics import confusion_matrix,recall_score
from sklearn import  metrics
from sklearn.linear_model import LogisticRegression

def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            return datain
        tb_name, col_name = column.split('.')
        new_cols.append((column, col_name))
    datain = datain.rename(columns=dict(new_cols))
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
    outdata = split_table_name(outdata)
    return outdata

curr_date = str(datetime.datetime.now())[0:10]
logging.info('当前日期: %s'%(curr_date))
curr_dir = '/home/souche/qiongjiu/hgc/'
curr_date=str(datetime.datetime.now())[0:10]
hive_cli = HiveCliHook("warehouse_hive")

DAYU_HOME : /home/souche/projects/datacenter-etl-v2


/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2021-02-03 17:09:05,603] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2021-02-03 17:09:05,629] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2021-02-03 17:09:06,582] {<ipython-input-2-c2e46d8a12df>:45} INFO - 当前日期: 2021-02-03


In [16]:
model_curr_date = '2021-02-02'
sql_1 = """
select t1.*,
       t2.guide_price/10000  guide_price  
from (select 
        model_code,
        first_license_plate_date,
        pay_time,
        real_pay_amount/10000 price,
        sale_type  
        from edw_dfc.dws_evt_trd_dfcf_ts_order_dd
        where ds = '"""+model_curr_date+"""') t1 
left join (select * from db_data.ods_car_model_model_price 
           where ds = '"""+model_curr_date+"""') t2 
on t1.model_code = t2.model_code
"""

dtype={'model_code':str}
dfc_data= read_from_hive2('mid_car_dfc_sale_order',sql_1,dtype)

model_columns = []
for col in dfc_data.columns:
    if len(col.split('.')) > 1:
        model_columns.append(col.split('.')[1])

    else:
        model_columns.append(col)
dfc_data.columns = model_columns 


[2021-02-03 17:13:53,694] {hiveserver2:138} INFO - Using database default as default
[2021-02-03 17:13:53,828] {hive_server_hook:112} INFO - Running query: 
select t1.*,
       t2.guide_price/10000  guide_price  
from (select 
        model_code,
        first_license_plate_date,
        pay_time,
        real_pay_amount/10000 price,
        sale_type  
        from edw_dfc.dws_evt_trd_dfcf_ts_order_dd
        where ds = '2021-02-02') t1 
left join (select * from db_data.ods_car_model_model_price 
           where ds = '2021-02-02') t2 
on t1.model_code = t2.model_code

[2021-02-03 17:14:28,331] {hive_server_hook:162} INFO - Written 10000 rows so far.
[2021-02-03 17:14:28,802] {hive_server_hook:162} INFO - Written 20000 rows so far.
[2021-02-03 17:14:29,277] {hive_server_hook:162} INFO - Written 30000 rows so far.
[2021-02-03 17:14:29,760] {hive_server_hook:162} INFO - Written 40000 rows so far.
[2021-02-03 17:14:30,199] {hive_server_hook:162} INFO - Written 50000 rows so far.
[2021-02

[2021-02-03 17:15:09,798] {hive_server_hook:162} INFO - Written 920000 rows so far.
[2021-02-03 17:15:10,238] {hive_server_hook:162} INFO - Written 930000 rows so far.
[2021-02-03 17:15:10,642] {hive_server_hook:162} INFO - Written 940000 rows so far.
[2021-02-03 17:15:11,075] {hive_server_hook:162} INFO - Written 950000 rows so far.
[2021-02-03 17:15:11,522] {hive_server_hook:162} INFO - Written 960000 rows so far.
[2021-02-03 17:15:11,959] {hive_server_hook:162} INFO - Written 970000 rows so far.
[2021-02-03 17:15:12,390] {hive_server_hook:162} INFO - Written 980000 rows so far.
[2021-02-03 17:15:12,791] {hive_server_hook:162} INFO - Written 990000 rows so far.
[2021-02-03 17:15:13,225] {hive_server_hook:162} INFO - Written 1000000 rows so far.
[2021-02-03 17:15:13,672] {hive_server_hook:162} INFO - Written 1010000 rows so far.
[2021-02-03 17:15:14,107] {hive_server_hook:162} INFO - Written 1020000 rows so far.
[2021-02-03 17:15:14,509] {hive_server_hook:162} INFO - Written 1030000 r

[2021-02-03 17:15:52,039] {hive_server_hook:162} INFO - Written 1890000 rows so far.
[2021-02-03 17:15:52,477] {hive_server_hook:162} INFO - Written 1900000 rows so far.
[2021-02-03 17:15:52,915] {hive_server_hook:162} INFO - Written 1910000 rows so far.
[2021-02-03 17:15:53,323] {hive_server_hook:162} INFO - Written 1920000 rows so far.
[2021-02-03 17:15:53,765] {hive_server_hook:162} INFO - Written 1930000 rows so far.
[2021-02-03 17:15:54,207] {hive_server_hook:162} INFO - Written 1940000 rows so far.
[2021-02-03 17:15:54,653] {hive_server_hook:162} INFO - Written 1950000 rows so far.
[2021-02-03 17:15:55,102] {hive_server_hook:162} INFO - Written 1960000 rows so far.
[2021-02-03 17:15:55,516] {hive_server_hook:162} INFO - Written 1970000 rows so far.
[2021-02-03 17:15:55,961] {hive_server_hook:162} INFO - Written 1980000 rows so far.
[2021-02-03 17:15:56,410] {hive_server_hook:162} INFO - Written 1990000 rows so far.
[2021-02-03 17:15:56,857] {hive_server_hook:162} INFO - Written 2

[2021-02-03 17:16:33,965] {hive_server_hook:162} INFO - Written 2860000 rows so far.
[2021-02-03 17:16:34,405] {hive_server_hook:162} INFO - Written 2870000 rows so far.
[2021-02-03 17:16:34,843] {hive_server_hook:162} INFO - Written 2880000 rows so far.
[2021-02-03 17:16:35,279] {hive_server_hook:162} INFO - Written 2890000 rows so far.
[2021-02-03 17:16:35,719] {hive_server_hook:162} INFO - Written 2900000 rows so far.
[2021-02-03 17:16:36,122] {hive_server_hook:162} INFO - Written 2910000 rows so far.
[2021-02-03 17:16:36,560] {hive_server_hook:162} INFO - Written 2920000 rows so far.
[2021-02-03 17:16:36,991] {hive_server_hook:162} INFO - Written 2930000 rows so far.
[2021-02-03 17:16:37,451] {hive_server_hook:162} INFO - Written 2940000 rows so far.
[2021-02-03 17:16:37,856] {hive_server_hook:162} INFO - Written 2950000 rows so far.
[2021-02-03 17:16:38,294] {hive_server_hook:162} INFO - Written 2960000 rows so far.
[2021-02-03 17:16:38,742] {hive_server_hook:162} INFO - Written 2

In [18]:
car_data_columns= ["country_id","manufacturer","battery_quality_mile","battery_quality_time","brand_code","brand_name","car_body",
"cylinder_number","driving_mode","engine_volume_l","fuel_form","gear_box_type","guide_price","height","import_type",
"intake_type","length","level","max_power","max_torque","model_code","model_name","quality_mile","quality_time","rate",
"rate_count","seat_number_top","series_code","series_level","series_name","wheel_base","width","year","down_market_time",
"production_time","shutdown_time","time_to_market","year_1","year_2","year_3","year_4","year_5","year_6","year_7","year_8",
"year_9","year_10","year_11","year_12","year_13","year_14","year_15","year_16"]

model = pd.read_csv(curr_dir+'all_car_model_sep_word_to_sql',sep='\t')
car_model = model[['model_code','series_code','brand_code']]



/home/souche/projects/calculation/py3dev_new/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
## 计算车龄月
dfc_data = dfc_data.loc[~dfc_data['first_license_plate_date'].isnull()]
dfc_data = dfc_data.loc[~dfc_data['pay_time'].isnull()]

dfc_data['license_time_year'] = dfc_data['first_license_plate_date'].map(lambda x:int(str(x).split('-')[0]))
dfc_data['publish_time_year'] = dfc_data['pay_time'].map(lambda x:int(str(x).split('-')[0]))

dfc_data['license_time_month'] = dfc_data['first_license_plate_date'].map(lambda x:int(str(x).split('-')[1]))
dfc_data['publish_time_month'] = dfc_data['pay_time'].map(lambda x:int(str(x).split('-')[1]))
dfc_data['license_month'] = dfc_data['publish_time_year'].map(lambda x:int(x)) * 12 + \
                                            dfc_data['publish_time_month'] - \
                                            dfc_data['license_time_year'].map(lambda x:int(x)) * 12 - \
                                            dfc_data['license_time_month']

In [20]:
dfc_data['canzhi'] = dfc_data['price']/dfc_data['guide_price'] 
dfc_data = dfc_data.rename(columns={'license_month':'month'})
dfc_data['month'] = dfc_data['month']

dfc_sales = dfc_data.loc[dfc_data['sale_type'] == 'RESALE'][['model_code','month','pay_time','canzhi']]
dfc_purchase = dfc_data.loc[dfc_data['sale_type'] == 'WHOLESALE'][['model_code','month','pay_time','canzhi']]


In [22]:
def merge_data(df1, df2, anchor):
    cols_to_use = df2.columns.difference(df1.columns)
    df2_reindex = df2.set_index(anchor)[cols_to_use].reset_index()
    return df1.merge(df2_reindex, on = anchor, how = 'left')

dfc_sales_cleaned = merge_data(dfc_sales, car_model, 'model_code')
dfc_purchase_cleaned = merge_data(dfc_purchase, car_model, 'model_code')
dfc_sales_cleaned = dfc_sales_cleaned[~dfc_sales_cleaned['month'].isna()]
dfc_purchase_cleaned = dfc_purchase_cleaned[~dfc_purchase_cleaned['month'].isna()]
dfc_sales_cleaned = dfc_sales_cleaned[~dfc_sales_cleaned['canzhi'].isna()]
dfc_purchase_cleaned = dfc_purchase_cleaned[~dfc_purchase_cleaned['canzhi'].isna()]

dfc_sales_cleaned['month'] = dfc_sales_cleaned['month'].astype(int)
dfc_purchase_cleaned['month'] = dfc_purchase_cleaned['month'].astype(int)
dfc_sales_cleaned['canzhi'] = dfc_sales_cleaned['canzhi'].astype(float)
dfc_purchase_cleaned['canzhi'] = dfc_purchase_cleaned['canzhi'].astype(float)

median_sale = dfc_sales_cleaned.groupby(by = ['series_code', 'month'])['canzhi'].median().reset_index()
median_sale.columns = ['series_code', 'age', 'sale_residual_median']
median_purchase = dfc_purchase_cleaned.groupby(by = ['series_code', 'month'])['canzhi'].median().reset_index()
median_purchase.columns = ['series_code', 'age', 'purchase_residual_median']

profit_margin = pd.merge(median_sale, median_purchase, how = 'inner', on = ['series_code', 'age'])
profit_margin['margin'] = profit_margin['sale_residual_median'] - profit_margin['purchase_residual_median']

series_profit_margin_stats = profit_margin.groupby(by = 'series_code')['margin'].median().rename('series_margin_median').reset_index()
temp1 = profit_margin.groupby(by = 'series_code')['margin'].std().rename('series_margin_std').reset_index()
temp2 = profit_margin.groupby(by = 'series_code')['series_code'].count().rename('data_counts').reset_index()
series_profit_margin_stats = pd.merge(series_profit_margin_stats, temp1, how = 'left', on = 'series_code')
series_profit_margin_stats = pd.merge(series_profit_margin_stats, temp2, how = 'left', on = 'series_code')

# 车型库去重
car_model_no_duplicate = car_model[['series_code', 'brand_code']].drop_duplicates()

# series level merge
all_profit_margin_stats_series = series_profit_margin_stats.merge(car_model_no_duplicate, on = 'series_code' , how = 'right')[['series_code','series_margin_median','brand_code','data_counts']]
all_profit_margin_stats_series['data_counts'].fillna(0, inplace = True)

all_profit_margin_stats_brand = all_profit_margin_stats_series.query('data_counts >= 10 and series_margin_median >0').groupby(
    by='brand_code')['series_margin_median'].median().rename('brand_margin_median').reset_index()

all_profit_margin_stats = pd.merge(
    all_profit_margin_stats_series, all_profit_margin_stats_brand, how='left', on='brand_code')

# brand_margin_median填充负值、空值以及counts小于10
all_profit_margin_stats.loc[(all_profit_margin_stats['series_margin_median'] <= 0) | (all_profit_margin_stats['data_counts'] <= 10), 'series_margin_median'] = all_profit_margin_stats.loc[(
    all_profit_margin_stats['series_margin_median'] <= 0) | (all_profit_margin_stats['data_counts'] <= 10), 'brand_margin_median']

# 全局中位数继续填充空值
global_median = all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'].notna(), 'series_margin_median'].median()
all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'].isna(), 'series_margin_median'] = global_median

# margin小于1%的填充为车系25分位点
temp = all_profit_margin_stats.groupby(by='brand_code')['series_margin_median'].quantile(0.25).rename('series_margin_median_25').reset_index()
all_profit_margin_stats = pd.merge(all_profit_margin_stats, temp, how='left', on='brand_code')
all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'] <= 0.01, 'series_margin_median'] = all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'] <= 0.01, 'series_margin_median_25']

# margin仍然小于1%的用global_median填充
all_profit_margin_stats.loc[all_profit_margin_stats['series_margin_median'] <= 0.01, 'series_margin_median'] = global_median

result = pd.merge(car_model, all_profit_margin_stats, how = 'left', on = 'series_code')
result = result[['model_code', 'series_margin_median']]
result.columns = ['model_code', 'margin_median']


In [23]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60741 entries, 0 to 60740
Data columns (total 2 columns):
model_code       60741 non-null object
margin_median    60741 non-null float64
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [25]:
ph = curr_dir+"enterprise2_retial_purchase_margin.csv"
result.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/全网数据-优化模型/"+curr_date+"/enterprise2_retial_purchase_margin.csv", ph)


[2021-02-03 17:17:43,150] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/全网数据-优化模型/2021-02-03/enterprise2_retial_purchase_margin.csv .


In [26]:
result.shape

(60741, 2)